In [4]:
from dataloader import flickr30k,collate_custom
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

## DataSet

In [9]:
transform_train = transforms.Compose([
            transforms.RandomCrop(224),
            transforms.RandomHorizontalFlip(), 
            transforms.ToTensor(), 
            transforms.Normalize((0.444, 0.421, 0.385), 
                                 (0.285, 0.277, 0.286))])

In [6]:
transform_test = transforms.Compose([
            transforms.CenterCrop(224),
            transforms.ToTensor(), 
            transforms.Normalize((0.444, 0.421, 0.385), 
                                 (0.285, 0.277, 0.286))])

In [7]:
batch_size = 2
shuffle_img = False
num_workers = 2

In [10]:
dataset_train = flickr30k(transform=transform_train,image_root="",ann_root="./flickr30k",split="train")
dataloader_train = DataLoader(dataset=dataset_train,batch_size=batch_size,shuffle=shuffle_img,collate_fn=collate_custom)

Using downloaded and verified file: ./flickr30k/flickr30k_train.json
